In [1]:
from datetime import *
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
beginDate = date(2017, 10, 17)
endDate = date(2018, 4, 11)

delta = endDate - beginDate

dateFormat = "%Y%m%d"

In [3]:
delta

datetime.timedelta(176)

In [4]:
# replaces weird 1/2 character with .5
# changes PK to 0
def __cleanLines(s):
    if ord(s[-1]) == 189:
        return s[:-1] + ".5"
    elif s == "PK":
        return "0"
    else:
        return s

In [12]:
def getSpreads(day):
    gameDivClasses = ["_3A-gC _2DWLf _3zKaX", "_3A-gC _2DWLf _3zKaX _1BrlL", "_3A-gC _2DWLf _3zKaX _1BrlL _1Mxxm", "_3A-gC _2DWLf _3zKaX _1Mxxm"]

    page = requests.get("https://www.sportsbookreview.com/betting-odds/nba-basketball/pointspread/?date=" + str(day))
    soup = BeautifulSoup(page.content, "lxml")
    
    games = []
    for c in gameDivClasses:
        games.extend(soup.find_all("div", class_=c))

    games_df = pd.DataFrame(columns=["date", "away", "home", "opening_spread"])
    for game in games:
        row = {"date": day}

        teams_html = game.find_all("span", class_="_3O1Gx")
        #print(teams_html)
        
        row["away"] = teams_html[0].get_text()
        row["home"] = teams_html[1].get_text()
        
        openingLines_html = game.find_all("span", class_="_3Nv_7")
        #print(openingLines_html)
        
        row["opening_spread"] = openingLines_html[1].get_text()
        row["opening_spread"] = __cleanLines(row["opening_spread"])
        
        games_df = games_df.append(row, ignore_index=True)
    
    return games_df

,date,away,home,opening_spread
0,20171018,Brooklyn,Indiana,-3.5
1,20171018,Miami,Orlando,+3
2,20171018,Milwaukee,Boston,-5.5
3,20171018,Atlanta,Dallas,-6.5
4,20171018,Minnesota,San Antonio,-1
5,20171018,Philadelphia,Washington,-7
6,20171018,Charlotte,Detroit,-3
7,20171018,New Orleans,Memphis,-3
8,20171018,Denver,Utah,-2
9,20171018,Houston,Sacramento,+8.5


In [6]:
def getMoneyLines(day):
    gameDivClasses = ["_3A-gC _2DWLf _3zKaX", "_3A-gC _2DWLf _3zKaX _1BrlL", "_3A-gC _2DWLf _3zKaX _1BrlL _1Mxxm", "_3A-gC _2DWLf _3zKaX _1Mxxm"]

    page = requests.get("https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=" + str(date))
    soup = BeautifulSoup(page.content, "lxml")

    games = []
    for c in gameDivClasses:
        games.extend(soup.find_all("div", class_=c))

    games_df = pd.DataFrame(columns=["date", "away", "away_ML", "home", "home_ML"])
    for game in games:
        row = {"date": day}

        teams_html = game.find_all("span", class_="_3O1Gx")
        row["away"] = teams_html[0].get_text()
        row["home"] = teams_html[1].get_text()

        lines_html = game.find_all("span", class_="_1QEDd")
        row["away_ML"] = lines_html[2].get_text()
        row["home_ML"] = lines_html[3].get_text()

        games_df = games_df.append(row, ignore_index=True)

    return games_df

In [7]:
spreads_df = pd.DataFrame()
ml_df = pd.DataFrame()
for i in range(delta.days + 1):
    date = (beginDate + timedelta(i)).strftime(dateFormat)
    print(date)
    
    spreads_df = spreads_df.append(getSpreads(date))
    ml_df = ml_df.append(getMoneyLines(date))

spreads_df = spreads_df
ml_df = ml_df

season_df = pd.merge(spreads_df, ml_df, how="outer", on=["date", "away", "home"]).reset_index(drop=True)
season_df

20171017
20171018
20171019
20171020
20171021
20171022
20171023
20171024
20171025
20171026
20171027
20171028
20171029
20171030
20171031
20171101
20171102
20171103
20171104
20171105
20171106
20171107
20171108
20171109
20171110
20171111
20171112
20171113
20171114
20171115
20171116
20171117
20171118
20171119
20171120
20171121
20171122
20171123
20171124
20171125
20171126
20171127
20171128
20171129
20171130
20171201
20171202
20171203
20171204
20171205
20171206
20171207
20171208
20171209
20171210
20171211
20171212
20171213
20171214
20171215
20171216
20171217
20171218
20171219
20171220
20171221
20171222
20171223
20171224
20171225
20171226
20171227
20171228
20171229
20171230
20171231
20180101
20180102
20180103
20180104
20180105
20180106
20180107
20180108
20180109
20180110
20180111
20180112
20180113
20180114
20180115
20180116
20180117
20180118
20180119
20180120
20180121
20180122
20180123
20180124
20180125
20180126
20180127
20180128
20180129
20180130
20180131
20180201
20180202
20180203
20180204
2

,date,away,home,opening_spread,away_ML,home_ML
0,20171017,Boston,Cleveland,-3,+155,-175
1,20171017,Houston,Golden State,-9,+385,-485
2,20171018,Brooklyn,Indiana,-3.5,+145,-165
3,20171018,Miami,Orlando,+3,-150,+130
4,20171018,Milwaukee,Boston,-5.5,+205,-245
5,20171018,Atlanta,Dallas,-6.5,+220,-260
6,20171018,Minnesota,San Antonio,-1,+105,-125
7,20171018,Philadelphia,Washington,-7,+255,-310
8,20171018,Charlotte,Detroit,-3,+145,-165
9,20171018,New Orleans,Memphis,-3,+135,-155


In [8]:
season_df.to_csv("lines_2017_2018.tsv", sep="\t", index=False)

In [ ]:
 # NOTE: manually remove all-star games and postponed IND @ NOP on Feb 7, 2018